In [24]:
import os
import sys
# sys.path.append("/app/routers/nlp/util")
sys.path.append("../nlp_util/")

from dothis_keyword import VBR, GensimRelated
from dothis_nlp import decode_and_convert
from ai_dataload import download_file_from_s3
from tqdm import tqdm
import math
import json
import pymysql
import pandas as pd
from datetime import datetime, timedelta

In [22]:

#### to temp
def vbr_data_collect():

    def get_dates_between(start_date, end_date):
        """
        두 날짜 사이의 모든 날짜를 리스트로 반환하되, 현재 날짜 이후의 날짜는 제외합니다.
        
        :param start_date: 시작 날짜 (YYYY-MM-DD 형식의 문자열)
        :param end_date: 종료 날짜 (YYYY-MM-DD 형식의 문자열)
        :return: 두 날짜 사이의 모든 날짜를 포함하는 리스트 (현재 날짜 이후 제외)
        """
        # 현재 날짜
        current_date = datetime.now().strftime('%Y%m%d')
        
        # 날짜 범위 생성
        dates = pd.date_range(start=start_date, end=end_date)
        
        # 날짜를 문자열 형식으로 변환하여 리스트로 반환, 현재 날짜 이후는 제외
        return [date.strftime('%Y%m%d') for date in dates if date.strftime('%Y%m%d') <= current_date]


    ##################################################################################
    # PROXMOX MYSQL 데이터베이스 연결 정보
    host = os.environ.get('MYSQL_HOST') # RDS 엔드포인트 URL 또는 IP 주소
    port = int(os.environ.get('MYSQL_PORT')) # RDS 데이터베이스 포트 (기본값: 3306)
    user = os.environ.get('MYSQL_USER') # MySQL 계정 아이디
    password = os.environ.get('MYSQL_PW') # MySQL 계정 비밀번호
    db_name = "dothis_pre" # 데이터베이스 이름

    # PROXMOX MYSQL 연결하기
    conn = pymysql.connect(host=host, 
                        port=port,
                        user=user, 
                        password=password, 
                        db=db_name, 
                        charset='utf8mb4')

    # 데이터베이스 커서(Cursor) 객체 생성
    cursor = conn.cursor()

    df = pd.DataFrame()
    end_limit = 30
    now_limit = 0
    end_total = 7
    now_total = 0
    time_date = 0

    while True:
        if end_total == now_total:
            break

        ### time_date까지 데이터가 채워지지 않았을 경우를 대비.
        if end_limit == now_limit:
            print(f"Data is not sufficiently populated. Collects only total {end_total} match data by {end_limit}.")
            # logger.info(f"Data is not sufficiently populated. Collects only total {end_total} match data by {end_limit}.")
            break

        start_date = datetime.now() - timedelta(days=time_date)
        start_date_str = start_date.strftime("%Y-%m-%d")
        _start_date_str = start_date.strftime("%Y%m%d")
        
        end_date = start_date + timedelta(days=7)
        _end_date_str = end_date.strftime("%Y%m%d")
        end_date_str = end_date.strftime("%Y-%m-%d")
        dates = get_dates_between(start_date_str, end_date_str)
        # print("start_date_str, end_date_str", start_date_str, end_date_str)
        # print("dates", dates)
        ############################ pre 데이터 가져오기
        # 테이블 존재 여부 확인 쿼리
        video_data_table = f"video_data_{_start_date_str}"
        check_query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{db_name}' AND table_name = '{video_data_table}';"
        cursor.execute(check_query)
        result = cursor.fetchone()
        if not result:
            now_limit += 1
            time_date += 1
            continue
        
        query = f"""
                desc {db_name}.{video_data_table};
                """
        cursor.execute(query)
        columns = [row[0] for row in cursor.fetchall()]
        
        query = f"""
                select * from {db_name}.{video_data_table};
                """
        cursor.execute(query)

        v_etc = pd.DataFrame(cursor.fetchall(), columns=columns)
        if len(v_etc) == 0:
            now_limit += 1
            time_date += 1
            continue
        ########################################################

        # 1번 13번 클러스터 제외
        v_etc = v_etc[~v_etc.video_cluster.isin([1, 13, "None"])]
        v_etc['video_published'] = pd.to_datetime(v_etc['video_published'])  # video_published 컬럼을 datetime 타입으로 변환
        v_etc.dropna(subset=['use_text'], inplace=True)
        
        # print(start_date_str,"~",end_date_str, " 시작")
        columns = ["video_id", "video_views", "video_performance", "YEAR", "MONTH", "DAY"]
        columns_to_str = ", ".join(columns)

        ########################### 히스토리 가져오기 to temp
        etc = pd.DataFrame()
        h_etc = pd.DataFrame()
        for date in dates:
            history_data_table = f"video_history_{date}"
            query = f"""
                        select {columns_to_str} from dothis_temp.{history_data_table};
                    """
            cursor.execute(query)
            _h_etc = pd.DataFrame(cursor.fetchall(), columns=columns)
            h_etc = pd.concat([h_etc, _h_etc], axis=0)
            
        etc = pd.merge(v_etc, h_etc, on='video_id', how='inner')            
        etc = etc.drop_duplicates()
        # print("len(etc)", len(etc)) 

        df = pd.concat([df, etc], axis=0)
        time_date += 1
        now_total  += 1
        print(f"Use {start_date_str} ({now_total}/{end_total})")
        # logger.info(f"Use {start_date_str} ({now_total}/{end_total})")

        ########################################################

    ############################ 조회수 계산하기
    # crawled_date를 datetime 형식으로 변환
    # null 값을 0으로 대체 및 소수점 이하 제거 및 정수형 변환
    df[["YEAR", "MONTH", "DAY"]] = df[["YEAR", "MONTH", "DAY"]].fillna(0).astype(int)
    df["crawled_date"] = pd.to_datetime(df['YEAR'].astype(str)+"-"+df['MONTH'].astype(str)+"-"+df['DAY'].astype(str), format="%Y-%m-%d", errors='coerce')
    df.sort_values(['crawled_date'], ascending=True, inplace=True)            
    df.reset_index(drop=True, inplace=True)   
    
    # 각 id에 대해 최신 날짜를 가진 행의 인덱스를 찾기
    idx = df.groupby('video_id')['crawled_date'].idxmax()

    # 인덱스를 이용해 최신 날짜 데이터를 필터링
    df = df.loc[idx]
    df = df.drop_duplicates()
    df.use_text = df.use_text.progress_apply(decode_and_convert)
    df.use_text = df.use_text.progress_apply(lambda x: " ".join([i[0] for i in x]))

    df.sort_values(['crawled_date'], ascending=True, inplace=True)            
    df.reset_index(drop=True, inplace=True)          

    return df

In [23]:
bucket_name = "dothis-ai"
s3_prefix = "models/related/"
# local_dir = f"/app/{s3_prefix}"
local_dir = "../../../testdata"
if not os.path.exists(local_dir):
    os.makedirs(local_dir)
    
df = vbr_data_collect()

Use 2024-08-31 (1/7)
Use 2024-08-30 (2/7)
Use 2024-08-29 (3/7)
Use 2024-08-28 (4/7)
Use 2024-08-27 (5/7)
Use 2024-08-26 (6/7)
Use 2024-08-25 (7/7)


100%|██████████| 42611/42611 [00:00<00:00, 392370.29it/s]


NameError: name 'download_file_from_s3' is not defined

In [26]:
download_file_from_s3(bucket_name=bucket_name,
                      s3_prefix=s3_prefix,
                      local_dir=local_dir)


Skipping directory ../../../testdata/
Downloaded models/related/related_model.bin to ../../../testdata/related_model.bin
Downloaded models/related/related_model.bin.syn1neg.npy to ../../../testdata/related_model.bin.syn1neg.npy
Downloaded models/related/related_model.bin.trainables.syn1neg.npy to ../../../testdata/related_model.bin.trainables.syn1neg.npy
Downloaded models/related/related_model.bin.wv.vectors.npy to ../../../testdata/related_model.bin.wv.vectors.npy
Downloaded models/related/related_model.bin.wv.vectors_ngrams.npy to ../../../testdata/related_model.bin.wv.vectors_ngrams.npy
Downloaded models/related/related_model.bin.wv.vectors_vocab.npy to ../../../testdata/related_model.bin.wv.vectors_vocab.npy


In [28]:
data_path = "../../../usedata"
gr_path = os.path.join(local_dir, "related_model.bin")
josa_path = os.path.join(data_path, "kor_josa.txt")
stopwords_path = os.path.join(data_path, "stopwords_for_related.txt")
model = "word2vec"

gr = GensimRelated(path=gr_path,
                   josa_path=josa_path,
                   stopwords_path=stopwords_path,
                   model=model)

vr = VBR(df,
        josa_path=josa_path,
        stopwords_path=stopwords_path)

Reading Stopwords List: 100%|██████████| 1907/1907 [00:00<00:00, 3310653.03it/s]


In [35]:

def scale_score_result(word, vbr_ratio=0.55, inference_ratio=0.45):
    related_inference = gr.gensim_related(word, split_word_check=True)
    vbr_related = vr.vbr_related(word, ranking=1, threadholds=5, df_len=10000)

    if (len(related_inference) > 0) and (len(vbr_related) > 0):
    
        vbr_score_scales = [(x / max(vbr_related.values())) * vbr_ratio for x in vbr_related.values()]
        inference_score_scales = [(x / max(related_inference.values())) * inference_ratio for x in related_inference.values()]
        # 업데이트
        vbr_related = dict(zip(vbr_related.keys(), vbr_score_scales))
        related_inference = dict(zip(related_inference.keys(), inference_score_scales))
        # 두 딕셔너리의 모든 연관어 모음
        all_related_word = list(set(vbr_related.keys()) | set(related_inference.keys()))
        
        ### dict.get(key, default)로 해당 연관어가 없으면 0이 반환되도록 함.
        combined_dict = dict()
        for related_word in all_related_word:
            vbr_score_value = vbr_related.get(related_word, 0)
            inference_score_value = related_inference.get(related_word, 0)
            
            combined_score = vbr_score_value + inference_score_value
            ### combined_score가 null값이거나 0이 아닌 경우에만 저장
            if (not math.isnan(combined_score)) and (combined_score != 0):
                combined_dict[related_word] = combined_score
        # 값을 기준으로 내림차순 정렬
        combined_dict = dict(sorted(combined_dict.items(), key=lambda item: item[1], reverse=True))
    
    elif (len(related_inference) > 0) and (len(vbr_related) == 0):                    
        inference_score_scales = [(x / max(related_inference.values())) * inference_ratio for x in related_inference.values()]
        related_inference = dict(zip(related_inference.keys(), inference_score_scales))
        # 값을 기준으로 내림차순 정렬
        combined_dict = dict(sorted(related_inference.items(), key=lambda item: item[1], reverse=True))
        
    elif (len(related_inference) == 0) and (len(vbr_related) > 0):                    
        vbr_score_scales = [(x / max(vbr_related.values())) * vbr_ratio for x in vbr_related.values()]
        vbr_related = dict(zip(vbr_related.keys(), vbr_score_scales))
        # 값을 기준으로 내림차순 정렬
        combined_dict = dict(sorted(vbr_related.items(), key=lambda item: item[1], reverse=True))
    else:
        combined_dict = dict()
    return combined_dict


In [37]:
word = "칸예"
print(scale_score_result(word, vbr_ratio=0.55, inference_ratio=0.45))

{'웨스트': 0.9085145628561144, '내한': 0.8386440612935728, '리스닝파티': 0.6599890596351143, '리스닝': 0.4846560222636259, '파티': 0.4846560222636259, 'west': 0.4774462866441829, '카니예': 0.45, '카녜': 0.40383379188127966, '런어웨이': 0.4027723910652595, 'Kanye': 0.3841926178140266, 'kanye': 0.3406707797317701, '컨버스': 0.2896510351122901}
